In [1]:
import pickle
import h5py
import matplotlib.pyplot as plt
import numpy as np
import seaborn
import pandas as pd
import random
import keras

Using TensorFlow backend.


In [2]:
with open('result/outer_cv.pkl', 'rb') as f:
    results = pickle.load(f)
with open('result/args.pkl', 'rb') as f:
    args = pickle.load(f)

In [3]:
def style_maximizing_table(df):
    g_pallete = seaborn.light_palette("green", as_cmap=True, reverse=True)
    return df.style.background_gradient(cmap=g_pallete)

In [7]:
scores = [r['score_values'] for r in results]
print(scores)
maes = [s['mean_absolute_error'] for s in scores]
stds = [s['std'] for s in scores]

result_df = pd.DataFrame.from_dict({
    "mean_absolute_error" : maes,
    "std": stds
})
style_maximizing_table(result_df)

[{'mean_absolute_error': 0.21727556343853294, 'std': 0.17220756176046179}]


,mean_absolute_error,std
0,0.217276,0.172208


In [5]:
data = h5py.File(args.path, 'r')

In [6]:
result_idx = 0
result = results[result_idx]
test_idx = sorted(random.sample(result['test_idx'].tolist(), 20))

X_test = data['X'][test_idx]
y_test = data['y'][test_idx]

model = keras.models.load_model('result/model%d.h5' % result_idx)
model.summary()

predicted_values = model.predict(X_test).squeeze().tolist()
true_values = y_test
result_df = pd.DataFrame.from_dict({
    "true" : true_values,
    "predicted" : predicted_values,
    "diff" : np.abs(np.array(predicted_values)-np.array(true_values))
})
style_maximizing_table(result_df)

InternalError: failed initializing StreamExecutor for CUDA device ordinal 0: Internal: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 12806062080

In [ ]:
# Plot history.
with open('result/history%d.pkl' % result_idx, 'rb') as f:
    history = pickle.load(f).history

val_loss = history['val_loss']
loss = history['loss']
print("min val_loss: %f" % min(val_loss))
print("min loss: %f" % min(loss))
gen_gap = np.array(val_loss)-np.array(loss)

fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
fig.set_size_inches(10, 12)
ax1.set_title("loss")
ax1.plot(loss)
ax2.set_title("val loss")
ax2.plot(val_loss)
ax3.set_title("generalization gap (val loss - loss)")
ax3.plot(gen_gap)